In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
#X_test, y_train, y_test 

In [3]:
X_all = X_train_in.append(X_val, ignore_index=True)

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(X_all.ДИСЦИПЛИНА)

X_train_in.loc[:,"type"] = le.transform(X_train_in.loc[:,"ДИСЦИПЛИНА"])
X_val.loc[:,"type"] = le.transform(X_val.loc[:,"ДИСЦИПЛИНА"])

In [5]:
X_train_in["mark"]=y_train_in["mark"]

In [8]:
stud_mean_mark = X_train_in.groupby("STD_ID").mean()["mark"]
X_train_in.loc[:,"stud_mean"] = X_train_in.loc[:,"STD_ID"].map(stud_mean_mark)

In [9]:
X_val.loc[:,"stud_mean"] = X_val.loc[:,"STD_ID"].map(stud_mean_mark)

In [10]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_train_in,X_train_in["mark"], shuffle=True, test_size=0.2)

In [13]:
y_test.shape, y_train.shape

((21012,), (84046,))

In [14]:
import lightgbm

In [17]:
train_col = ["STD_ID","КУРС","СЕМЕСТР","stud_mean","type"]
X_train["STD_ID"]= X_train["STD_ID"].astype(int)
X_test["STD_ID"]= X_test["STD_ID"].astype(int)


/home/rumata/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/rumata/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
lgb = lightgbm.LGBMRegressor(n_estimators=1000, verbose=1)

In [19]:
lgb.fit(X_train[train_col],y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
              verbose=1)

In [20]:
from sklearn.metrics import mean_absolute_error, r2_score

In [21]:
mean_absolute_error(y_test, lgb.predict(X_test[train_col])), r2_score(y_test, lgb.predict(X_test[train_col]))

(0.4026296142766824, 0.537483716394414)

In [23]:
y_pred = pd.read_csv("sample_submission.csv",index_col=0)
y_pred["mark"] = lgb.predict(X_val[train_col])

In [24]:
y_pred.to_csv("baseline_submission.csv")